# DeepEval demo en Binder (deepeval_demo.ipynb)

Este cuaderno muestra dos rutas:
1) **Sin API keys** (usando `JsonCorrectnessMetric`, no requiere LLM).
2) **Con API key** (usando métricas con LLM como `AnswerRelevancyMetric` y `GEVal`).

> **Sugerencia**: Si estás en Binder, este repo debe incluir un `requirements.txt` (ya preparado) para que el entorno se construya automáticamente.

In [ ]:

# === Instalación/Comprobación de dependencias ===
import importlib, sys, subprocess

def pip_install(*packages):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', '-U', *packages])

try:
    import deepeval  # noqa
except Exception:
    pip_install('deepeval', 'python-dotenv')

print('Entorno listo ✅')


## Parte A · Evaluación **sin API keys** (JsonCorrectnessMetric)
`JsonCorrectnessMetric` valida que el `actual_output` cumpla un **esquema JSON** (Pydantic). 
No utiliza un LLM, por lo que funciona sin configurar claves.

In [ ]:

from pydantic import BaseModel, RootModel
from typing import List
from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import JsonCorrectnessMetric

# Definimos el esquema esperado
class Item(BaseModel):
    name: str
    price: float

class ItemList(RootModel[List[Item]]):
    pass

metric = JsonCorrectnessMetric(
    expected_schema=ItemList,
    include_reason=False,   # no usamos LLM para la 'reason'
    strict_mode=True        # exige coincidencia exacta de esquema
)

# Dos casos de prueba: uno correcto y otro incorrecto
ok_output = '[{"name": "Cuaderno", "price": 3.5}, {"name": "Bolígrafo", "price": 1.2}]'
ko_output = '{"name": "Cuaderno", "price": "tres"}'  # NO es lista y price no es float

test_cases = [
    LLMTestCase(input='Devuélveme una lista de artículos', actual_output=ok_output),
    LLMTestCase(input='Devuélveme una lista de artículos', actual_output=ko_output),
]

# Ejecutamos la evaluación
_ = evaluate(test_cases=test_cases, metrics=[metric])


## Parte B · Métricas con **LLM** (requiere API key)
Las métricas como `AnswerRelevancyMetric` o `GEVal` usan un **LLM como juez**. 
Por defecto DeepEval utiliza modelos de OpenAI; configura `OPENAI_API_KEY` (o usa otro proveedor soportado) antes de ejecutar.

In [ ]:

# OPCIONAL: Define tu API key en variables de entorno (sólo a modo de ejemplo).
# En Binder puedes ejecutar esta celda manualmente y pegar tu clave.
import os
os.environ.get('OPENAI_API_KEY') or os.environ.update({'OPENAI_API_KEY': 'PON_AQUI_TU_OPENAI_API_KEY'})
print('OPENAI_API_KEY configurada:', bool(os.environ.get('OPENAI_API_KEY')))


In [ ]:

# Ejemplo con AnswerRelevancyMetric + GEval
from deepeval.metrics import AnswerRelevancyMetric, GEval
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval import evaluate

answer_relevancy = AnswerRelevancyMetric(
    threshold=0.7,
    model='gpt-4.1',      # puedes cambiar a otro modelo soportado
    include_reason=True,
)

correctness = GEval(
    name='Correctness',
    criteria=(
        """Determina si el 'actual output' es correcto con respecto al 'expected output'.
        Usa criterio estricto para respuestas numéricas o hechos.
        """
    ),
    evaluation_params=[
        LLMTestCaseParams.ACTUAL_OUTPUT,
        LLMTestCaseParams.EXPECTED_OUTPUT,
    ],
    threshold=0.5,
)

tc = LLMTestCase(
    input="What if these shoes don't fit?",
    actual_output='We offer a 30-day full refund at no extra cost.',
    expected_output="You're eligible for a free full refund within 30 days of purchase.",
)

_ = evaluate([tc], [answer_relevancy, correctness])


---
### Notas
- Para ejecutar métricas con LLM **sin OpenAI**, revisa la documentación de modelos soportados (Anthropic, Gemini, Ollama, etc.) y pasa `model=...` al crear la métrica.
- Este cuaderno está preparado para ejecutarse en Binder si tu repo incluye `requirements.txt`.